In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd
import numpy as np
import tabula
import os

***DATA EXTRACTION***

In [2]:
# WEB SCRAPPING TABLE (df_salario):

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)
driver.get('https://portal.trt3.jus.br/internet/servicos/valores/salario-minimo')
url = driver.find_element(By.XPATH,'//*[@id="parent-fieldname-text"]/div/table')
html = url.get_attribute('outerHTML')
driver.quit()
df_salario = pd.read_html(html)[0]

In [3]:
# WEB SCRAPPING TABLE FROM PDF (margens_2013):

margens_2013 = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2013.pdf', pages = 'all')
margens_2013 = pd.concat(margens_2013)

In [4]:
# WEB SCRAPPING TABLE FROM PDF (margens_2023):

margens = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2023.pdf', pages = 'all')
margens_2023 = pd.concat(margens)

In [5]:
#READING XLSX FILES IN DATAFRAME (brasil):

brasil = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\brasil_semanal.xlsx')

In [6]:
#READING XLSX FILES IN DATAFRAME (estados):

estados = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\estados_semanal.xlsx')

***EXPLORING AND CLEANING THE DATA***

*dataframe df_salario:*

In [7]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.head()

,ANO/VIGÊNCIA,VALOR,MOEDA,FONTE
0,2023 1º/1/2023,"R$ 1.302,00 Por Mês R$ 43,40 Por Dia R$ 5,92 P...",Real,"MP n. 1.143, de 12/12/2022 DOU 12/12/2022 - Ed..."
1,- 2022 1º/1/2022,"R$ 1.212,00 Por Mês R$ 40,40 Por Dia R$ 5,51 P...",Real,"MP n. 1.091, de 30/12/2021 DOU 31/12/2021 (Con..."
2,2021 1º/1/2021,"R$ 1.100,00 Por Mês R$ 36,67 Por Dia R$ 5,00 P...",Real,"MP n. 1.021, de 30/12/2020 DOU 31/12/20 (Conve..."
3,2020 1º/2/2020,"R$ 1.045,00 Por Mês R$ 34,83 Por Dia R$ 4,75 P...",Real,"MP n. 919, de 30/1/2020 DOU 31/1/20"
4,2020 1º/1/2020,"R$ 1.039,00 Por Mês R$ 34,63 Por Dia R$ 4,72 P...",Real,"MP n. 916, de 31/12/2019 DOU 31/12/19 - Edição..."


In [8]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.tail()

,ANO/VIGÊNCIA,VALOR,MOEDA,FONTE
129,1954 4/7/1954,"Cr$ 2.200,00 Por Mês (valor regional para BH e...",Cruzeiro,"Decreto n.35.450, de 1º/5/1954 DOU 4/5/54"
130,1952 1º/1/1952,"Cr$ 900,00 Por Mês Cr$ 30,00 Por Dia Cr$ 3,75 ...",Cruzeiro,"Decreto n.30.342, de 24/12/1951 DOU 26/12/51"
131,1943 1º/12/1943,"Cr$ 270,00 Por Mês Cr$ 10,80 Por Dia Cr$ 1,35 ...",Cruzeiro,"Decreto-Lei n. 5.977, de 10/11/1943 DOU 22/11/43"
132,1943 17/7/1943,"Cr$ 212,00 Por Mês Cr$ 8,48 Por Dia Cr$ 1,06 P...",Cruzeiro,"Decreto-Lein.5.670, de 15/7/1943 DOU 17/7/43"
133,1940 4/7/1940,170$000 Por Mês 6$800 Por Dia $850 Por Hora (v...,Real,"Decreto-Lein.2.162, de 1º/5/1940 DOU 4/5/40 (D..."


In [9]:
#EXPLORING THE df_salario DATAFRAME:

df_salario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ANO/VIGÊNCIA  134 non-null    object
 1   VALOR         134 non-null    object
 2   MOEDA         134 non-null    object
 3   FONTE         134 non-null    object
dtypes: object(4)
memory usage: 4.3+ KB


In [10]:
#PERFORMING TREATMENT IN DATAFRAME df_salario USING DE FUNCTION BELOW:
    
def limpar_df_salario(df):
    df = df.drop(columns = ['MOEDA','FONTE']).drop(index=range(12,134))
    df = df.rename(columns = {'ANO/VIGÊNCIA':'DATA', 'VALOR':'VALOR POR MÊS'})
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.slice(stop=11)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace('P','').str.replace(',','.')
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$ +', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$\xa0', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'\xa0', '', regex=True)
    df['DATA'] = df['DATA'].str.replace(r'\s+\d{1,2}º/\d{1,2}/\d{4}', '', regex=True).str.replace('- ','')
    df = df.set_index('DATA')
    return df






In [11]:
# NEW DATAFRAME TREATED (salario_minimo):

salario_minimo = limpar_df_salario(df_salario)

*dataframe margens_2013*

In [12]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_2013.head()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brasil,Preço de Realização do Produtor,"11,33","11,34","11,34","11,34","11,34","11,34",NaN,"11,34",...,"11,34",NaN,"11,34",NaN,"11,33",NaN,NaN,NaN,NaN,NaN
2,NaN,CIDE,-,-,-,-,-,-,NaN,-,...,-,NaN,-,NaN,-,NaN,NaN,NaN,NaN,NaN
3,NaN,PIS/COFINS,"2,18","2,18","2,18","2,18","2,18","2,18",NaN,"2,18",...,"2,18",NaN,"2,18",NaN,"2,18",NaN,NaN,NaN,NaN,NaN
4,NaN,Preço do Produtor s/ ICMS c/ CIDE/PIS/COFINS,"13,51","13,52","13,52","13,52","13,52","13,52",NaN,"13,52",...,"13,52",NaN,"13,52",NaN,"13,51",NaN,NaN,NaN,NaN,NaN


In [13]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_2013.tail()

,Unnamed: 0,Unnamed: 1,jan/13,fev/13,mar/13,abr/13,mai/13,jun/13,Unnamed: 2,jul/13,...,out/13,Unnamed: 4,nov/13,Unnamed: 5,dez/13,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea ""b"" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa ""Auxílio-Gás"" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa ""Bolsa-Família"" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas."
2,Roraima\r(alíquota de ICMS\rdo produtor)\r17%,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"10,81\r-\r2,18\r12,99\r182,06%\r184,11%\r7,53\...","10,85\r-\r2,18\r13,03\r183,52%\r180,76%\r7,53\...","10,78\r-\r2,18\r12,96\r190,67%\r191,64%\r7,73\...","10,88\r-\r2,18\r13,06\r188,84%\r189,00%\r7,73\...","10,79\r-\r2,18\r12,97\r190,74%\r190,79%\r7,73\...","10,83\r-\r2,18\r13,01\r189,49%\r190,31%\r7,73\...",NaN,"10,80\r-\r2,18\r12,98\r190,09%\r190,97%\r7,73\...",...,"10,81\r-\r2,18\r12,99\r190,21%\r190,79%\r7,73\...",NaN,"10,86\r-\r2,18\r13,04\r188,23%\r190,06%\r7,73\...",NaN,"10,83\r-\r2,18\r13,01\r189,43%\r190,26%\r7,73\...",NaN,NaN,NaN,NaN,NaN
0,Santa Catarina\r(alíquota de ICMS\rdo produtor...,Preço de Realização do Produtor\rCIDE\rPIS/COF...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...","11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",...,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,"11,69\r-\r2,18\r13,87\r173,74%\r173,74%\r5,18\...",NaN,NaN,NaN,NaN,NaN
1,São Paulo\r(alíquota de ICMS\rdo pr

In [14]:
#EXPLORING THE margens_2013 DATAFRAME:

margens_2013.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 0
Data columns (total 23 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [24]:
#PERFORMING TREATMENT IN DATAFRAME df_salario USING DE FUNCTION BELOW:

estado = margens_2013.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
estado = estado.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
estado = estado.reset_index(drop=True).drop(index=(0)



SyntaxError: unexpected EOF while parsing (1470081950.py, line 6)

*dataframe brasil*

In [ ]:
#EXPLORING THE brasil DATAFRAME:
brasil.head()

In [ ]:
#EXPLORING THE brasil DATAFRAME:
brasil.tail()

In [ ]:
#EXPLORING THE brasil DATAFRAME:
brasil.info()

In [ ]:
#PERFORMING TREATMENT IN DATAFRAME brasil USING DE FUNCTION BELOW:

def limpar_df_br(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime
    df = df.rename(columns = {'AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP':'DATA INICIAL','Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'PRODUTO','Unnamed: 3':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 4':'UNIDADE DE MEDIDA','Unnamed: 5':'PREÇO MÉDIO REVENDA','Unnamed: 6':'DESVIO PADRÃO REVENDA','Unnamed: 7':'PREÇO MÍNIMO REVENDA','Unnamed: 8':'PREÇO MÁXIMO REVENDA','Unnamed: 9':'MARGEM MÉDIA REVENDA','Unnamed: 10':'COEF DE VARIAÇÃO REVENDA','Unnamed: 11':'PREÇO MÉDIO DISTRIBUIÇÃO','Unnamed: 12':'DESVIO PADRÃO DISTRIBUIÇÃO','Unnamed: 13':'PREÇO MÍNIMO DISTRIBUIÇÃO','Unnamed: 14':'PREÇO MÁXIMO DISTRIBUIÇÃO','Unnamed: 15':'COEF DE VARIAÇÃO DISTRIBUIÇÃO'})
    df_1 = df.drop(index=range(0, 17)).drop(columns=['DATA INICIAL', 'DESVIO PADRÃO REVENDA', 'MARGEM MÉDIA REVENDA','COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO',	'PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO'])
    df_limpo = df_1.loc[df_1['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo.index.name = 'DATA'
    return df_limpo


In [ ]:
# NEW DATAFRAME TREATED (glp_br):
glp_br = limpar_df_br(brasil)

***DATAFRAME estado***

In [ ]:
#EXPLORING THE (estados) DATAFRAME:
estados.head()

In [ ]:
#EXPLORING THE (estados) DATAFRAME:
estados.tail()

In [ ]:
#EXPLORING THE (estados) DATAFRAME:
estados.info()

In [ ]:
#PERFORMING TREATMENT IN DATAFRAME estados USING DE FUNCTION BELOW:

def limpar_df_estado(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime   
    df = df.drop(index=range(0, 17)).drop(columns = ['AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP', 'Unnamed: 8', 'Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17'])
    estados_limpo = df.rename(columns = {'Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'REGIÃO','Unnamed: 3':'ESTADO','Unnamed: 4':'PRODUTO','Unnamed: 5':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 6':'UNIDADE DE MEDIDA','Unnamed: 7':'PREÇO MÉDIO REVENDA','Unnamed: 9':'PREÇO MÍNIMO REVENDA','Unnamed: 10':'PREÇO MÁXIMO REVENDA'})
    df_limpo = estados_limpo.loc[estados_limpo['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo = df_limpo.set_index('DATA FINAL')
    df_limpo.index.name = 'DATA'
    return df_limpo


In [ ]:
# NEW DATAFRAME TREATED (glp_regioes):
glp_regioes = limpar_df_estado(estados)